In [1]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "last_expr"

In [2]:
import re, string, unicodedata
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import inflect
import pandas as pd
# nltk.download('stopwords')

In [3]:
f = open("dataset/passage_collection_new.txt", 'r', encoding = 'utf-8') 

In [4]:
document = f.read()

In [5]:
header_list = ["qid", "pid", "query", "passage"]
candidate_passages_top1000 = pd.read_csv("dataset/candidate_passages_top1000.tsv", sep='\t', names=header_list)

In [6]:
header_list = ["qid", "query"]
test_queries = pd.read_csv("dataset/test-queries.tsv", sep='\t', names=header_list)

# Preprocessing

## Lowercase

In [24]:
document = document.lower()

## Tokenizing

In [25]:
# nltk.download('punkt')
tokens = nltk.word_tokenize(document)


## Remove Punctuation

In [9]:
def remove_punctuation(tokens):
    new_tokens = []
    for token in tokens:
        new_token = re.sub(r'[^\w\s]', '', token)
        if new_token != '':
            new_tokens.append(new_token)
    return new_tokens

In [26]:
tokens = remove_punctuation(tokens)

## Remove Stopwords

In [11]:
def remove_stopwords(tokens):
    new_tokens = []
    stopword_set = set(stopwords.words('english'))
    for token in tokens:
        if token not in stopword_set:
            new_tokens.append(token)
    return new_tokens


In [27]:
# nltk.download('stopwords')
tokens = remove_stopwords(tokens)

## Lemmatization

In [13]:
def lemmatize_verbs(tokens):
    lemmatizer = WordNetLemmatizer()
    root_words = []
    for token in tokens:
        root_word = lemmatizer.lemmatize(token, pos='v')
#         root_word = lemmatizer.lemmatize(token, pos='n')
#         root_word = lemmatizer.lemmatize(token, pos='a')
        root_words.append(root_word)
    return root_words

In [28]:
# nltk.download('wordnet')
tokens = lemmatize_verbs(tokens)

## Remove numbers

In [15]:
def remove_numbers(tokens):
    """Replace all interger occurrences in list of tokenized words with textual representation"""
    new_tokens = []
    for token in tokens:
        if token.isdigit():
#             new_token = num2words(int(token), to = 'ordinal')
#             new_tokens.append(new_token)
            pass
        else:
            new_tokens.append(token)
    return new_tokens


In [29]:
tokens = remove_numbers(tokens)

In [16]:
def preprocessing(passage):
    passage = passage.lower()
    tokens = nltk.word_tokenize(passage)
    tokens = remove_punctuation(tokens)
    tokens = remove_stopwords(tokens)
    tokens = lemmatize_verbs(tokens)
    tokens = remove_numbers(tokens)
    return tokens

# Inverted Index

## Extracting Pid and Passages

In [18]:
candidate_passages_top1000

,qid,pid,query,passage
0,494835,7130104,"sensibilities, definition",This is the definition of RNA along with examp...
1,1128373,7130104,iur definition,This is the definition of RNA along with examp...
2,131843,7130104,definition of a sigmet,This is the definition of RNA along with examp...
3,20455,7130335,ar glasses definition,Best Answer: The AR designation comes from the...
4,719381,7130335,what is ar balance,Best Answer: The AR designation comes from the...
...,...,...,...,...
189872,1056204,79980,who was the first steam boat operator,Other operators with special formats accept mo...
189873,1132213,7998257,how long to hold bow in yoga,You may be surprised that to learn that yoga t...
189874,324211,7998651,how much money a united airline get as a capta...,Find cheap airline tickets & deals on flights ...
189875,1116341,7998709,closed ended mortgage definition,"What is a wrap-around mortgage, and who is it ..."


In [19]:
passage_dict = {}
for idx, row in candidate_passages_top1000.iterrows():
    pid = row['pid']
    passage = preprocessing(row['passage'])
    if pid not in passage_dict:
        passage_dict[pid] = passage
#         passage_dict[pid] = None
#     passage_dict[pid] = passage

    

In [40]:
len(passage_dict)

182469

In [30]:
inverted_index = {}

# term_frequency = frequency_dict[term].frequency if term in frequency_dict else 0

In [31]:
len(tokens)

5889367

In [41]:
tokens_no_dup = set(tokens)
len(tokens_no_dup)

172464

In [33]:
passage = passage_dict[7130104]
freqDist = nltk.FreqDist(passage)
passage
freqDist
for check in freqDist:
    print(check)

if 'rna' in freqDist:
    print(freqDist['rna'])

rna
definition
along
examples
type
molecules
5


In [48]:
def get_vocabulary(data):
    tokens = []
    for token_list in data.values():
        tokens = tokens + token_list
#     print("tokens:", tokens)
    fdist = nltk.FreqDist(tokens)
#     print("fdist:", fdist.items())
    return list(fdist.keys())

In [ ]:
check = get_vocabulary(passage_dict)

In [ ]:
len(check)

In [35]:
for token in tokens_no_dup:
    for pid, passage in passage_dict.items():
        freqDist = nltk.FreqDist(passage)
        if token in freqDist:
            inverted_index.setdefault(token, [])
            inverted_index[token].append((pid, freqDist[token]))

            

KeyboardInterrupt: 

In [47]:
inverted_index

{'littoral': [(3738049, 1), (3738049, 1)],
 'pcstransforming': [(1929915, 1), (1929915, 1)],
 'stylespecific': [(1784170, 1), (1784170, 1)],
 'registrant': [(5299766, 2),
  (3177247, 1),
  (7464871, 2),
  (7954264, 2),
  (2281565, 2),
  (5979419, 1),
  (5299771, 2),
  (4750062, 1),
  (6583273, 1),
  (4587381, 1),
  (135740, 1),
  (5299767, 1),
  (3555076, 2),
  (2230084, 1),
  (8661772, 1),
  (4134040, 1),
  (5299766, 2),
  (3177247, 1),
  (7464871, 2),
  (7954264, 2),
  (2281565, 2),
  (5979419, 1),
  (5299771, 2),
  (4750062, 1),
  (6583273, 1),
  (4587381, 1),
  (135740, 1),
  (5299767, 1),
  (3555076, 2),
  (2230084, 1),
  (8661772, 1),
  (4134040, 1)],
 'swiftly': [(1824485, 1),
  (7899946, 1),
  (8778148, 1),
  (5896290, 1),
  (8420551, 1),
  (6327503, 1),
  (6610325, 1),
  (1824485, 1),
  (7899946, 1),
  (8778148, 1),
  (5896290, 1),
  (8420551, 1),
  (6327503, 1),
  (6610325, 1)],
 'n565': [(1493230, 2),
  (5740859, 3),
  (2607134, 1),
  (93612, 2),
  (4016314, 4),
  (5740854, 